necessary packages

In [1]:
import numpy as np
from ipywidgets import interact
import plotly.graph_objects as go
from scipy.special import gamma
from scipy.integrate import quad as integrate

define function for updating the layout of the figure:
* updates the layout of a plotly figure with predefined settings
* the **title** dict includes all title properties: **'x'** specifies the position of the title along the x-axis, **'xanchor'** specifies the anchor point of the title along the x-axis

In [2]:
def update_layout_of_graph(fig: go.Figure, title: str = 'Plot') -> go.Figure:
    fig.update_layout(
        width=1000,
        height=500,
        autosize=False,
        plot_bgcolor='black',
        title={'text': title, 'x': 0.5, 'xanchor': 'center'},  # dict includes all title properties; specifies the position of the title along the x-axis
        xaxis_title='t / s',
        yaxis_title='U / V',
        legend=dict(yanchor="top", y=0.9, xanchor="right", x=0.95),
        xaxis=dict(showline=True, linewidth=1, linecolor='black'),
        yaxis=dict(showline=True, linewidth=1, linecolor='black')
    )
    return fig

define function for creating a line plot:
* creates a Plotly scatter trace representing a line plot
* **x_lines** x-coordinates of the points on the line with an empty default array
* **y_lines** y-coordinates of the points on the line with an empty default array

In [3]:
def line_plot(x_lines: np.array = np.array([]), y_lines: np.array = np.array([]), title: str = 'predicted function',
show_legend: bool = True, visible: bool = True, ) -> go.Scatter:

    scatter = go.Scatter(x=x_lines, y=y_lines, line=dict(color="orange", width=3), name=title, showlegend= show_legend, visible=visible)
    return scatter

define function for a scatter plot:
* creates a scatter plot with Plotly
* **x_dots** is an array of x-coordinates for the data points in the scatter plot with an empty default array
* **y_dots** is an array of y-coordinates for the data points in the scatter plot with an empty default array

In [4]:
def scatter_plot(visible: bool = True, x_dots: np.array = np.array([]), y_dots: np.array = np.array([]), name_dots: str = 'observed points',
show_legend: bool = True, color: str = 'red') -> go.Scatter:

    scatter = go.Scatter(x=x_dots, y=y_dots, visible=visible, mode='markers', name=name_dots, marker=dict(color=color, size=8), showlegend=show_legend)
    return scatter

define function for plotting the uncertainty area:
* generates an area plot to represent the uncertainty using the Plotly
* **x_lines** an array of x-coordinates for the lines outlining the area plot with an empty default array
* **y_lower** an array of y-coordinates representing the lower boundary of the uncertainty area plot with an empty default array
* **y_upper** an array of y-coordinates representing the upper boundary of the uncertainty area plot with an empty default array

In [5]:
def uncertainty_area_plot(visible: bool = True, x_lines: np.array = np.array([]), y_lower: np.array = np.array([]), y_upper: np.array = np.array([]), name: str = 'mean plus/minus standard deviation') -> go.Scatter:

    scatter = go.Scatter(x=np.concatenate((x_lines, x_lines[::-1])), y=np.concatenate((y_upper, y_lower[::-1])), visible=visible, fill='toself', fillcolor='rgba(189, 195, 199, 0.5)', line=dict(color='rgba(200, 200, 200, 0)'), hoverinfo='skip', showlegend=True, name=name)
    return scatter

define squared exponential kernel:
* **variable1** & **variable2** correspond to t & t'
* **sigma** describes the average distance away from the function mean and l determines the reach of influence on neighbors
* **sigma** & **length** is assigned the float standard value 1
* **variable_1** & **variable_2** expect an float value

In [6]:
class SquaredExponentialKernel:
    def __init__(self, sigma: float = 1, length: float = 1, alpha: float = 0.75):
        self.sigma = sigma
        self.length = length
        self.alpha = alpha

    def __call__(self, tau_1: float, tau_2: float):
        def kernel(variable_1, variable_2):
            k_SE = float(self.sigma**2 * np.exp(-(np.linalg.norm(variable_1 - variable_2)**2) /(2 * self.length**2)))
            return k_SE
        return kernel(tau_1, tau_2)

visualize the squared exponential kernel:
* only for testing the squared exponential kernel and monitor the effect of changing the parameters
* **@interact** generates a user interface which allows for an interactive regulation of the parameters length, sigma and variable_2 in the defined ranges
* the function **update** is called whenever the values of the sliders are changed
* **batch_update()** ensures that all updates to the figure within the block are batched together

In [7]:
x_lines = np.arange(-10, 10, 0.1)
kernel = SquaredExponentialKernel()  # initialize an instance of the class SquaredExponentialKernel, length and sigma have already standard values
data = line_plot(x_lines=x_lines, y_lines=np.array([kernel(x, 0) for x in x_lines]))  # generates the data for the line distribution

fig = go.FigureWidget(data)  # generates an interactive plot-widget with the passed data
fig = update_layout_of_graph(fig, title='squared exponential kernel')

@interact(length=(0.1, 3, 0.1), sigma=(0.1, 3, 0.1), variable_2=(-10, 10, 0.1))
def update(length=1, sigma=1, variable_2=0):
    with fig.batch_update():
        kernel = SquaredExponentialKernel(sigma=sigma, length=length)  # a new instance of the class SquaredExponentialKernel with the updated length and sigma value is generated
        fig.data[0].y = np.array([kernel(x, variable_2) for x in x_lines])  # updates the y-values plot data, data can contain multiply scatter plots
        # with data[0].y the y-coordinates of the first scatter plot are accessed

fig

interactive(children=(FloatSlider(value=1.0, description='length', max=3.0, min=0.1), FloatSlider(value=1.0, d…

FigureWidget({
    'data': [{'line': {'color': 'orange', 'width': 3},
              'name': 'predicted function',
              'showlegend': True,
              'type': 'scatter',
              'uid': '18760880-cc2a-4eb1-bd27-ae9e5de9d21a',
              'visible': True,
              'x': array([-1.00000000e+01, -9.90000000e+00, -9.80000000e+00, -9.70000000e+00,
                          -9.60000000e+00, -9.50000000e+00, -9.40000000e+00, -9.30000000e+00,
                          -9.20000000e+00, -9.10000000e+00, -9.00000000e+00, -8.90000000e+00,
                          -8.80000000e+00, -8.70000000e+00, -8.60000000e+00, -8.50000000e+00,
                          -8.40000000e+00, -8.30000000e+00, -8.20000000e+00, -8.10000000e+00,
                          -8.00000000e+00, -7.90000000e+00, -7.80000000e+00, -7.70000000e+00,
                          -7.60000000e+00, -7.50000000e+00, -7.40000000e+00, -7.30000000e+00,
                          -7.20000000e+00, -7.10000000e+00, -7.000000

functions for creating the covariance matrix, each of them were implemented separately in such a way as to make them comprehensible with the notes, which is why they are referenced:
* **val_1** & **val_2** are equal to a certain element of the arrays **t** & **t'**
* **alpha**, **R** & **C** are set to a specific value

In [8]:
def k_fu(val_1, val_2, cov_func, R=2, C=0.5):
    alpha = cov_func.alpha
    l = cov_func.length
    b = 1/(R*C) * cov_func(val_1, val_2)

    def integrand(tau):
        a = - 1/gamma(1-alpha) * 1/(val_1 - tau)**alpha * 1/l**2 * (tau - val_2)
        return a * cov_func(tau, val_2)

    integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-9, epsrel=1e-9, limit=200)
    return integral_val + b

In [9]:
def k_uf(val_1, val_2, cov_func, R=2, C=0.5):
    alpha = cov_func.alpha
    l = cov_func.length
    b = 1/(R*C) * cov_func(val_1, val_2)

    def integrand(tau):
        a = 1/gamma(1-alpha) * 1/(val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
        return a * cov_func(val_1, tau)

    integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-9, epsrel=1e-9, limit=200)
    return integral_val + b

In [10]:
def k_uu(val_1, val_2, cov_func):
    return cov_func(val_1, val_2)

In [11]:
def k_ff(val_1, val_2, cov_func, R=2, C=0.5):
    alpha = cov_func.alpha
    b = 1/(R*C)*(k_fu(val_1, val_2, cov_func, R, C) - k_uu(val_1, val_2, cov_func))
    c = 1/(R*C)*(k_uf(val_1, val_2, cov_func, R, C) - k_uu(val_1, val_2, cov_func))
    d = 1/(R*C)**2*k_uu(val_1, val_2, cov_func)

    def B(val_1, val_2, tau):
        l = cov_func.length
        a = 1/gamma(1-alpha) * 1/(val_2 - tau)**alpha * 1/l**2 * (val_1 - tau)
        return a * cov_func(val_1, tau)

    def A(val_1, val_2):
        integrand = lambda tau: B(val_1, val_2, tau)
        integral_val, _ = integrate(integrand, 0, val_2, epsabs=1e-8, epsrel=1e-8, limit=200)
        return integral_val

    def derivative_A(tau, val_2, h= 0.01*np.sqrt(2e-53)):

        def A_tau(tau):
            return A(tau, val_2)
        dB = (A_tau(tau + h) - A_tau(tau - h)) / (2*h)
        return dB

    def double_integral(val_1, val_2):
        integrand = lambda tau: 1/gamma(1 - alpha) * 1/(val_1 - tau)**alpha * derivative_A(tau, val_2)
        integral_val, _ = integrate(integrand, 0, val_1, epsabs=1e-8, epsrel=1e-8, limit=200)
        return integral_val

    e = double_integral(val_1, val_2)

    return b + c + d + e

testing the implemented functions above:

In [12]:
val_1 = 0.1
val_2 = 0.1
x = np.arange(0.01, 3, 0.01)
cov_func = SquaredExponentialKernel()

for i in range(len(x)):
    kuf = k_uf(x[i], x[i], cov_func)
    kfu = k_fu(x[i], x[i], cov_func)
    kuu = k_uu(x[i], x[i], cov_func)
    kff = k_ff(x[i], x[i], cov_func)
    print('kuf = ', kuf, 'kuu = ', kuu, 'kfu = ', kfu, 'kff = ', kff)

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_5561/1571718703.py:14: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



kuf =  1.0006977511487722 kuu =  1.0 kfu =  1.0006977511487722 kff =  1.0013955022975445
kuf =  1.0016594455240448 kuu =  1.0 kfu =  1.0016594455240448 kff =  1.0033188910480897
kuf =  1.0027544526935086 kuu =  1.0 kfu =  1.0027544526935086 kff =  1.0055089053870172
kuf =  1.0039459382488531 kuu =  1.0 kfu =  1.0039459382488531 kff =  1.0078918764977063
kuf =  1.0052144997687686 kuu =  1.0 kfu =  1.0052144997687686 kff =  1.0104289995375373
kuf =  1.0065478296617367 kuu =  1.0 kfu =  1.0065478296617367 kff =  1.0130956593234735
kuf =  1.0079372916868248 kuu =  1.0 kfu =  1.0079372916868248 kff =  1.0158745833736496
kuf =  1.0093764205896734 kuu =  1.0 kfu =  1.0093764205896734 kff =  1.0187528411793467
kuf =  1.0108601512231887 kuu =  1.0 kfu =  1.0108601512231887 kff =  1.0217203024463775
kuf =  1.012384378582134 kuu =  1.0 kfu =  1.012384378582134 kff =  1.024768757164268
kuf =  1.0139456870460355 kuu =  1.0 kfu =  1.0139456870460355 kff =  1.027891374092071
kuf =  1.0155411739888554

a function for creating the sub and covariance matrix as described in the notes:
* works only correctly for squared matrices [n1 x n2] where n1 = n2

In [13]:
def create_sub_matrix(variable_1, variable_2, specified_cov_function, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    sub_matrix = np.zeros((n1, n2))

    for i, val_1 in enumerate(variable_1):
        for j, val_2 in enumerate(variable_2):
            sub_matrix[i, j] = specified_cov_function(val_1, val_2, cov_function)

    return sub_matrix

a function for creating the covariance matrix as described in the notes:
* works only correctly for squared matrices [2*n1 x 2*n2] where n1 = n2

In [14]:
def create_cov_matrix(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    cov = np.zeros((2*n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)
    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(2*n1):
        for j in range(2*n2):

            if i <= n1 - 1 and j <= n2 - 1:
                #print('kuu')
                cov[i, j] = kuu_sub_matrix[i, j]
            elif i <= n1 - 1 and n2 <= j <= 2*n2 - 1:
                #print('kuf')
                cov[i, j] = kuf_sub_matrix[i, j - n2]
            elif n1 <= i <= 2*n1 - 1 and j <= n2 - 1:
                #print('kfu')
                cov[i, j] = kfu_sub_matrix[i - n1, j]
            else:
                #print('kff')
                cov[i, j] = kff_sub_matrix[i - n1, j - n2]

    return cov

testing the implemented functions for creating the covariance matrix above

In [15]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 1, 1, 1]
#variable_1 = np.array([0.3, 0.8, 1.4, 1.8, 2])
#variable_2 = np.array([0.3, 0.8, 1.4, 1.8, 2])

cov_func = SquaredExponentialKernel()
cov = create_cov_matrix(variable_1, variable_2, cov_func)

print('covariance matrix = ', cov)

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_5561/1571718703.py:14: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



covariance matrix =  [[1.         1.         1.         1.         1.18406551 1.18406551
  1.18406551 1.18406551]
 [1.         1.         1.         1.         1.18406551 1.18406551
  1.18406551 1.18406551]
 [1.         1.         1.         1.         1.18406551 1.18406551
  1.18406551 1.18406551]
 [1.         1.         1.         1.         1.18406551 1.18406551
  1.18406551 1.18406551]
 [1.18406551 1.18406551 1.18406551 1.18406551 1.36813102 1.36813102
  1.36813102 1.36813102]
 [1.18406551 1.18406551 1.18406551 1.18406551 1.36813102 1.36813102
  1.36813102 1.36813102]
 [1.18406551 1.18406551 1.18406551 1.18406551 1.36813102 1.36813102
  1.36813102 1.36813102]
 [1.18406551 1.18406551 1.18406551 1.18406551 1.36813102 1.36813102
  1.36813102 1.36813102]]


define function for calculating q_u transposed

In [16]:
def q_u_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_u_trans = np.zeros((n1, 2*n2))

    kuu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uu, cov_function)
    kuf_sub_matrix = create_sub_matrix(variable_1, variable_2, k_uf, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_u_trans[i, j] = kuu_sub_matrix[i, j]
            else:
                q_u_trans[i, j] = kuf_sub_matrix[i, j - n2]

    return q_u_trans

In [17]:
def q_f_transposed(variable_1, variable_2, cov_function):
    n1 = len(variable_1)
    n2 = len(variable_2)
    q_f_trans = np.zeros((n1, 2*n2))

    kfu_sub_matrix = create_sub_matrix(variable_1, variable_2, k_fu, cov_function)
    kff_sub_matrix = create_sub_matrix(variable_1, variable_2, k_ff, cov_function)

    for i in range(n1):
        for j in range(2*n2):

            if j <= n2 - 1:
                q_f_trans[i, j] = kfu_sub_matrix[i, j]
            else:
                q_f_trans[i, j] = kff_sub_matrix[i, j - n2]

    return q_f_trans

testing if the function q_u_transposed works properly

In [18]:
variable_1 = [1, 1, 1, 1]
variable_2 = [1, 2, 1, 1]
cov_func = SquaredExponentialKernel()

q_u_trans = q_u_transposed(variable_1, variable_2, cov_func)
print('q_u transposed = ', q_u_trans)

q_u transposed =  [[1.         0.60653066 1.         1.         1.18406551 0.0923652
  1.18406551 1.18406551]
 [1.         0.60653066 1.         1.         1.18406551 0.0923652
  1.18406551 1.18406551]
 [1.         0.60653066 1.         1.         1.18406551 0.0923652
  1.18406551 1.18406551]
 [1.         0.60653066 1.         1.         1.18406551 0.0923652
  1.18406551 1.18406551]]


training: page 113 in GP for ML, please work please

In [19]:
def log_marginal_likelihood(data_x, data_y, cov_func, noise):
    n = len(data_x)
    y = np.concatenate([data_y, data_y])
    y_transposed = y.T
    K = create_cov_matrix(data_x, data_x, cov_func) + noise * np.identity(2*n)
    K_inverse = np.linalg.inv(K)

    a = n/2 * np.log(2*np.pi)  # normalization constant
    b = 1/2 * np.log(np.linalg.norm(K))  # complexity penalty
    c = 1/2 * np.dot(y, np.dot(K_inverse, y_transposed))  # involving observed targets
    log_ml = c + b + a  # is actually the negative log marginal likelihood
    return log_ml

sigma_vals = np.arange(1, 3.5, 0.05)
length_vals = np.arange(2.7, 4.5, 0.05)
memory = {'sigma': 1, 'length': 1, 'negative_log_marginal_likelihood': 1000}

data_x = np.array([0.29, 0.796, 1.394])
data_y = np.array([0.747, 0.447, 0.248])

for sigma_index, sigma_val in enumerate(sigma_vals):
    for l_index, length_val in enumerate(length_vals):
        cov_func = SquaredExponentialKernel(sigma=sigma_val, length=length_val)
        negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
        print('likelihood = ', negative_log_marginal_likelihood)

        if negative_log_marginal_likelihood < memory['negative_log_marginal_likelihood']:
            memory['length'] = length_val
            memory['sigma'] = sigma_val
            memory['negative_log_marginal_likelihood'] = negative_log_marginal_likelihood

        print('obtained sigma and l values that minimize the negative log marginal likelihood: ', 'sigma = ', memory['sigma'], 'length = ', memory['length'])

/var/folders/db/wdxh9ynd7_bfr2nwqswb2nn80000gn/T/ipykernel_5561/1571718703.py:14: IntegrationWarning:

The integral is probably divergent, or slowly convergent.



likelihood =  4.137355328672803
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  1.0 length =  2.7
likelihood =  4.134944140771177
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  1.0 length =  2.75
likelihood =  4.1321907457233324
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  1.0 length =  2.8
likelihood =  4.129106540893659
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  1.0 length =  2.8499999999999996
likelihood =  4.125701116753283
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  1.0 length =  2.8999999999999995
likelihood =  4.121982279122967
obtained sigma and l values that minimize the negative log marginal likelihood:  sigma =  1.0 length =  2.9499999999999993
likelihood =  4.117956099149511
obtained sigma and l values that minimize the negative log marginal likelihood:  

KeyboardInterrupt: 

In [ ]:
cov_func = SquaredExponentialKernel(sigma=1.7, length=3.5)
negative_log_marginal_likelihood = log_marginal_likelihood(data_x, data_y, cov_func, noise = 0.01)
print(negative_log_marginal_likelihood)

define class GPR:
* **data_x** is the input of the observed points
* **data_y** is the output of the observed points
* **noise** is the noise in the observed data
* **memory** is a variable where the mean, the covariance matrix and the variance is stored
* **3e-7** add this value to the noise in order to ensure stability when inverting the covariance matrix

In [ ]:
class GPR:
    def __init__(self, data_x, data_y, cov_func= SquaredExponentialKernel(sigma=memory['sigma'], length=memory['length']), noise=0.01):
        print('sigma memory = ', memory['sigma'])
        print('length memory = ', memory['length'])
        self.data_x = data_x
        self.data_y = np.concatenate([data_y, data_y])
        self.cov_func = cov_func
        self.noise = noise
        self.memory = None

        n = len(data_x)
        self.inv_cov_matrix_of_input_data = np.linalg.inv(create_cov_matrix(data_x, data_x, cov_func) + (noise + 3e-7)**2 * np.identity(2*n))

    def predict(self, at_values):
        q_u_trans = q_u_transposed(at_values, self.data_x, self.cov_func)
        q_f_trans = q_f_transposed(at_values, self.data_x, self.cov_func)

        # methode paper:
        mean_at_values_u = np.dot(q_u_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()
        #mean_at_values_f = np.dot(q_f_trans, np.dot(self.data_y, self.inv_cov_matrix_of_input_data.T).T).flatten()

        covariance_matrix_u = create_sub_matrix(at_values, at_values, k_uu, self.cov_func) - np.dot(q_u_trans, np.dot(self.inv_cov_matrix_of_input_data, q_u_trans.T))
        #covariance_matrix_f = create_sub_matrix(at_values, at_values, k_ff, self.cov_func) - np.dot(q_f_trans, np.dot(self.inv_cov_matrix_of_input_data, q_f_trans.T))

        variance_u = np.abs(np.diag(covariance_matrix_u))
        #variance_f = np.abs(np.diag(covariance_matrix_f))

        self.memory = {'mean': mean_at_values_u, 'covariance_matrix':covariance_matrix_u, 'variance': variance_u}

        return mean_at_values_u

defining function for plotting the GPR:
* **data_x** is a list of observed input data points
* **data_y** is a list of observed output data points
* **x** is equivalent to **at_values** in the GPR, the values we want to do predictions on

In [ ]:
def plot_GPR(data_x, data_y, model, x, data_x_mv, data_y_mv, visible=True) -> list:
    mean = model.predict(x)
    standard_deviation = np.sqrt(model.memory['variance'])
    data = []

    for i in range(1, 4):
        data.append(uncertainty_area_plot(x_lines=x, y_lower=mean -i * standard_deviation, y_upper=mean + i*standard_deviation, name=f'mean plus/minus {i}*standard deviation', visible=visible))

    data.append(scatter_plot(x_dots=data_x, y_dots=data_y, visible=visible, color='red'))
    data.append(scatter_plot(x_dots=data_x_mv, y_dots=data_y_mv, visible=visible, color='blue', name_dots='validation points'))
    data.append(line_plot(x_lines=x, y_lines=mean, visible=visible))
    return data

initializing the training data

In [ ]:
x_values = np.array([0.29, 0.796, 1.394])  # 1.798
y_values = np.array([0.747, 0.447, 0.248])  # 0.166

x_values_mv = np.array([0.0, 0.0183, 0.036, 0.055, 0.073, 0.091, 0.109, 0.127, 0.145, 0.164, 0.182, 0.2, 0.218, 0.236, 0.255, 0.273, 0.309, 0.327, 0.345, 0.364, 0.382, 0.4, 0.418, 0.436, 0.455, 0.473, 0.491, 0.509, 0.527, 0.545, 0.564, 0.582, 0.6, 0.618, 0.636, 0.655, 0.673, 0.69, 0.709, 0.727, 0.745, 0.764, 0.782, 0.818, 0.836, 0.855, 0.873, 0.891, 0.909, 0.927, 0.945, 0.964, 0.983, 1.0, 1.02, 1.036, 1.055, 1.073, 1.092, 1.109, 1.127, 1.145, 1.164, 1.182, 1.2, 1.215, 1.229, 1.248, 1.266, 1.284, 1.303, 1.321, 1.339, 1.358, 1.376, 1.413, 1.431, 1.45, 1.468, 1.486, 1.505, 1.523, 1.541, 1.56, 1.578, 1.596, 1.615, 1.633, 1.651, 1.67, 1.688, 1.706, 1.725, 1.743, 1.761, 1.78, 1.798, 1.817, 1.835, 1.853, 1.872, 1.89, 1.908, 1.927, 1.945, 1.963, 1.982, 1.996])
y_values_mv = np.array([1.0, 0.982, 0.964, 0.946, 0.928, 0.915, 0.897, 0.881, 0.866, 0.85, 0.834, 0.819, 0.803, 0.79, 0.774, 0.761, 0.734, 0.72, 0.707, 0.694, 0.68, 0.669, 0.658, 0.644, 0.633, 0.622, 0.611, 0.590, 0.588, 0.577, 0.568, 0.557, 0.548, 0.537, 0.526, 0.512, 0.508, 0.499, 0.49, 0.479, 0.47, 0.461, 0.452, 0.438, 0.43, 0.423, 0.414, 0.407, 0.4 ,0.394, 0.385, 0.378, 0.374, 0.369, 0.358, 0.351, 0.345, 0.34, 0.336, 0.327, 0.32, 0.315, 0.309, 0.304, 0.302, 0.298, 0.293, 0.286, 0.282, 0.277, 0.271, 0.266, 0.262, 0.257, 0.253, 0.244, 0.239, 0.235, 0.23, 0.226, 0.221, 0.219, 0.215, 0.21, 0.206, 0.204, 0.199, 0.195, 0.192, 0.188, 0.186, 0.181, 0.179, 0.174, 0.172, 0.168, 0.166, 0.163, 0.159, 0.157, 0.154, 0.152, 0.148, 0.145, 0.143, 0.141, 0.139, 0.136])

x = np.arange(0.01, 2, 0.01)

plot the output of the GPR

In [ ]:
# bocchi love you so much
model = GPR(x_values, y_values)
data = plot_GPR(data_x=x_values, data_y=y_values, x=x, model=model, data_x_mv=x_values_mv, data_y_mv=y_values_mv)
fig4 = go.Figure(data)
fig4 = update_layout_of_graph(fig=fig4, title=f'GPR with length {model.cov_func.length}, sigma {model.cov_func.sigma}, alpha {cov_func.alpha} and noise {model.noise}')
fig4.show()
fig4.write_html('/Users/simonjamnik/PycharmProjects/GaussianProcess/RC_0.75.html')